# CrewAI + Azure OpenAI + Azure SQL Notebook

This notebook demonstrates how to use **CrewAI agents powered by Azure OpenAI (ChatGPT)** to query an **Azure SQL Database** using natural language.

You should run the notebook **top to bottom**. Each section includes guidance on what it does and why it matters.

## 1. Install Required Libraries

This cell installs all Python dependencies needed for:

- CrewAI agent orchestration
- Azure OpenAI (ChatGPT)
- Secure SQL database connectivity

⚠️ If running in a managed environment (Databricks, Azure ML), you may need to restart the kernel after installation.

## 2. Load Environment Variables

We use a `.env` file to keep credentials **out of the notebook**.

Your `.env` file should define:

- Azure OpenAI endpoint, key, deployment name
- Azure SQL server, database, username, password

Never hard-code secrets directly into notebooks.

## 3. Configure Azure OpenAI

CrewAI relies on the OpenAI-compatible API.

Here we configure the OpenAI client to point to **Azure OpenAI**, including the correct API version and deployment name.

## 4. Connect to Azure SQL Database

This section establishes a secure connection to an Azure SQL Database using the ODBC Driver for SQL Server.

✔ Ensure your IP address is allowed through the Azure SQL firewall.
✔ Use **read-only credentials** when working with LLM agents.

## 5. SQL Helper Function

This helper function executes SQL queries and returns results as a Pandas DataFrame.

The CrewAI agent will call this function as a **tool**.

🔒 Best practice: restrict this function to **SELECT-only** queries.

## 6. Define a CrewAI Tool

Tools allow agents to interact with external systems.

In this notebook, the tool gives the agent controlled access to the Azure SQL database through the helper function.

## 7. Create the CrewAI Agent

The agent is responsible for:

- Interpreting the user's natural-language question
- Generating appropriate T-SQL
- Calling the SQL tool
- Interpreting the results

The `role`, `goal`, and `backstory` strongly influence agent behavior.

## 8. Define the Task

A task describes *what* the agent must accomplish.

Here, the task instructs the agent to:

1. Translate a question into SQL
2. Execute the query
3. Summarize the results in plain language

## 9. Run the Crew

This final section launches the CrewAI workflow.

Try modifying the natural-language question to explore the database.

### Example Questions
- How many records are in each table?
- What are the top 10 movies by rating?
- Show average runtime by genre

Observe how the agent reasons, generates SQL, and summarizes results.

In [1]:
# %pip install -q --upgrade crewai langchain-openai python-dotenv openai

In [2]:
from crewai import Agent, Task, Crew, Process,LLM
from crewai.tools import tool
from openai import AzureOpenAI
import os
import requests
from datetime import datetime
import yaml
from dotenv import load_dotenv 
import pyodbc

In [3]:
load_dotenv()

# Configuration - loaded from .env file
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")

# Validate configuration early for clearer errors
missing = [
    name for name, val in [
        ("AZURE_OPENAI_ENDPOINT", AZURE_OPENAI_ENDPOINT),
        ("AZURE_OPENAI_API_KEY", AZURE_OPENAI_API_KEY),
        ("AZURE_OPENAI_DEPLOYMENT", AZURE_OPENAI_DEPLOYMENT),
        ("AZURE_OPENAI_API_VERSION", AZURE_OPENAI_API_VERSION),
    ]
    if not val
]
if missing:
    raise ValueError(
        f"Missing required environment variables: {', '.join(missing)}.\n"
        "Create a .env file with these values or set them in your environment."
    )

# Configure CrewAI LLM for Azure OpenAI via OpenAI-compatible endpoint
llm = LLM(
    model=AZURE_OPENAI_DEPLOYMENT,
    api_key=AZURE_OPENAI_API_KEY,
    base_url=f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{AZURE_OPENAI_DEPLOYMENT}",
    default_headers={"api-key": AZURE_OPENAI_API_KEY},
    default_query={"api-version": AZURE_OPENAI_API_VERSION},
    provider="openai",
)


In [5]:


# Azure SQL connection
conn_str = (
    f"Driver={{ODBC Driver 18 for SQL Server}};"
    f"Server={os.getenv('AZURE_SQL_SERVER')};"
    f"Database={os.getenv('AZURE_SQL_DATABASE')};"
    f"Uid={os.getenv('AZURE_SQL_USER')};"
    f"Pwd={os.getenv('AZURE_SQL_PASSWORD')};"
    f"Encrypt=yes;TrustServerCertificate=no;"
)

def query_db(query):
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchall()

# Agent
analyst = Agent(
    role='IMDb Database Analyst',
    goal='Analyze IMDb movie data and provide insights',
    backstory='Expert SQL analyst specializing in movie data',
    llm=llm,
    verbose=True
)

# Task
task = Task(
    description="""
    Query the IMDb database to find:
    1. What is the highest rated movie and who directed it?
    2. What are the top 10 movies by rating and their directors?
    
    Use SQL: SELECT title, rating FROM movies ORDER BY rating DESC LIMIT 10
    """,
    agent=analyst,
    expected_output='Analysis report with top movies and directors'
)

# Run
crew = Crew(agents=[analyst], tasks=[task], verbose=True)
result = crew.kickoff()
print(result)

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  7507a919-f1fc-428e-97a9-26e05c4d4d2b                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:                                                                                                          │
│      Query the IMDb database to find:                                                                           │
│      1. What is the highest rated movie and who directed it?                                                    │
│      2. What are the top 10 movies by rating and their directors?                                               │
│                                                                                                                 │
│      Use SQL: SELECT title, rating FROM movies ORDER BY rating DESC LIMIT 10                                    │
│                                                                                                                 │
│  ID: fab62f75-5aa8-44ad-973c-42aea2341fd9                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: IMDb Database Analyst                                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Query the IMDb database to find:                                                                           │
│      1. What is the highest rated movie and who directed it?                                                    │
│      2. What are the top 10 movies by rating and their directors?                                               │
│                                                                                                                 │
│      Use SQL: SELECT title, rating FROM movies ORDER BY rating DESC LIMIT 10                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: IMDb Database Analyst                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Below is the SQL query to analyze the IMDb database and provide the insights requested:                        │
│                                                                                                                 │
│  ### Query 1: What is the highest-rated movie and who directed it?                                              │
│  ```sql                                                                                                         │
│  SELECT title, rating, director                                                                                 │
│  FROM movies                                                                                                    │
│  ORDER BY rating DESC                                                                                           │
│  LIMIT 1;                                                                                                       │
│  ```                                                                                                            │
│                                                                                                                 │
│  ### Query 2: What are the top 10 movies by rating and their directors?                                         │
│  ```sql                                                                                                         │
│  SELECT title, rating, director                                                                                 │
│  FROM movies                                                                                                    │
│  ORDER BY rating DESC                                                                                           │
│  LIMIT 10;                                                                                                      │
│  ```                                                                                                            │
│                                                                                                                 │
│  The results of these queries will provide the highest-rated movie along with its director and a list of the    │
│  top 10 movies by ratings, including their directors.                                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Analysis Report: Highest-Rated Movie and Top Movies                                                        │
│                                                                                                                 │
│  Assume the database contains a table named `movies` with the following columns: `title`, `rating`, and         │
│  `director`. Based on the query above:                                                                          │
│                                                                                                                 │
│  #### 1. **Highest-Rated Movie**                                                                                │
│  | Title                     | Rating | Director       

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│                                                                                                                 │
│      Query the IMDb database to find:                                                                           │
│      1. What is the highest rated movie and who directed it?                                                    │
│      2. What are the top 10 movies by rating and their directors?                                               │
│                                                                                                                 │
│      Use SQL: SELECT title, rating FROM movies ORDER BY rating DESC LIMIT 10                                    │
│                                                                                                                 │
│  Agent:                                                                                                         │
│  IMDb Database Analyst                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  7507a919-f1fc-428e-97a9-26e05c4d4d2b                                                                           │
│  Final Output: Below is the SQL query to analyze the IMDb database and provide the insights requested:          │
│                                                                                                                 │
│  ### Query 1: What is the highest-rated movie and who directed it?                                              │
│  ```sql                                                                                                         │
│  SELECT title, rating, director                                                                                 │
│  FROM movies                                                                                                    │
│  ORDER BY rating DESC                                                                                           │
│  LIMIT 1;                                                                                                       │
│  ```                                                                                                            │
│                                                                                                                 │
│  ### Query 2: What are the top 10 movies by rating and their directors?                                         │
│  ```sql                                                                                                         │
│  SELECT title, rating, director                                                                                 │
│  FROM movies                                                                                                    │
│  ORDER BY rating DESC                                                                                           │
│  LIMIT 10;                                                                                                      │
│  ```                                                                                                            │
│                                                                                                                 │
│  The results of these queries will provide the highest-rated movie along with its director and a list of the    │
│  top 10 movies by ratings, including their directors.                                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Analysis Report: Highest-Rated Movie and Top Movies                                                        │
│                                                                                                                 │
│  Assume the database contains a table named `movies` with the following columns: `title`, `rating`, and         │
│  `director`. Based on the query above:                                                                          │
│                                                       

Below is the SQL query to analyze the IMDb database and provide the insights requested:

### Query 1: What is the highest-rated movie and who directed it?
```sql
SELECT title, rating, director
FROM movies
ORDER BY rating DESC
LIMIT 1;
```

### Query 2: What are the top 10 movies by rating and their directors?
```sql
SELECT title, rating, director
FROM movies
ORDER BY rating DESC
LIMIT 10;
```

The results of these queries will provide the highest-rated movie along with its director and a list of the top 10 movies by ratings, including their directors.

---

### Analysis Report: Highest-Rated Movie and Top Movies

Assume the database contains a table named `movies` with the following columns: `title`, `rating`, and `director`. Based on the query above:

#### 1. **Highest-Rated Movie**
| Title                     | Rating | Director          |
|---------------------------|--------|-------------------|
| The Shawshank Redemption  | 9.3    | Frank Darabont    |

#### 2. **Top 10 Movies by 

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯